In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from datetime import date
from datetime import datetime
from datetime import time
from datetime import timedelta
from os import listdir
from os.path import isfile, join
import re
from shapely.geometry import Point
from shapely.geometry import LineString
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
import mplleaflet
import math


##### IMPORTANT METHODS DEFINITION #####

def get_angle(lon1, lat1, lon2, lat2):
    """
    Returns angle between the coordinates
    """
    lon_diff = lon2 - lon1
    lat_diff = lat2 - lat1
    return np.arctan2(lat_diff, lon_diff)

def get_angle_degrees(lon1, lat1, lon2, lat2):
    return np.degrees(get_angle(lon1, lat1, lon2, lat2))

def get_angle_points(pt1: Point, pt2: Point):
    """
    Returns angle between two points
    http://wikicode.wikidot.com/get-angle-of-line-between-two-points
    x is lon, y is lat
    """
    lon_diff = pt2.x - pt1.x
    lat_diff = pt2.y - pt1.y
    return math.degrees(math.atan2(lat_diff, lon_diff))

def get_angle_line_string(line_string: LineString):
    """
    Returns the angle between the first two points in a linestring
    """
    return get_angle(line_string[0],line_string[1])

def points_to_linestring(points):
    """
    The first given point will be the first in order
    """
    points_tuple_list = []
    for point in points:
        points_tuple_list.append((point.x, point.y))
    return LineString(points_tuple_list)

def parse_brazilian_datetime(dt_series):
    return pd.datetime.strptime(str(dt_series), '%d/%m/%Y %H:%M:%S')

# IMPORTANT VARIABLES DEFINITION

folder_path = 'G:/onibus-curitiba-2018-10/'
chosen_bus_lines = ['022']
chosen_vehicles = ['LR802']
gps_datum = {'init': 'epsg:4326'} # Distance in degrees
sirgas_utm_22s_datum = {'init': 'epsg:31982'} # Distance in meters
buffer_radius = 30
time_dif_seconds_threshold = 600
fig = None

##### CHECKING FILES IN FILESYSTEM #####

all_files = [f for f in listdir(folder_path) if isfile(join(folder_path, f))]

tables = [
    'linhas',
    'pontosLinha',
    'shapeLinha',
    'tabelaVeiculo',
    'veiculos'
]

files = pd.DataFrame({'file_name':all_files})
files['file_path'] = files.apply(lambda row: folder_path + row['file_name'], axis=1)
files['date'] = files.apply(lambda row: np.datetime64(re.search(r'[0-9]{4}_[0-9]{2}_[0-9]{2}',row['file_name']).group().replace('_','-'), 'D'), axis=1)
files['file_type'] = files.apply(lambda row: re.search(r'(.+_)([a-zA-Z]+)(\.json\.xz)',row['file_name']).group(2), axis=1)
files = files.astype({'file_path':'category','file_type':'category','file_name':'category'})
files.sort_values(by=['file_type','date',],inplace=True)

files_filtered_by_date = files[(files['date'] >= '2018-10-02') & (files['date'] <= '2018-10-02')]
files_filtered_by_date.set_index(
    keys=['file_type','date'],
    drop=False,
    append=False,
    inplace=True,
    verify_integrity=True
)
files_filtered_by_date

In [ ]:
##### VEICULOS #####

columns_rename = {
    'COD_LINHA': 'line_id',
    'DTHR': 'date_time',
    'LAT': 'lat',
    'LON': 'lon',
    'VEIC': 'vehicle_id'
}

columns_types = {
    'lat': 'float64',
    'lon': 'float64',
    'vehicle_id': 'category'
}

filtered_files = files_filtered_by_date[files_filtered_by_date['file_type'] == 'veiculos']

df_list = []

print("Using the following files:")
for index, row in filtered_files.iterrows():
    file = row['file_path']
    print(file)
    date = row['date']
    df = pd.read_json(lines=True,path_or_buf=file)
    print('Length before filtering {}'.format(len(df.index)))
    df['COD_LINHA'] = df['COD_LINHA'].apply(str)
    df = df[df['COD_LINHA'].isin(chosen_bus_lines)]
    df = df[df['VEIC'].isin(chosen_vehicles)]
    df['date'] = date
    df_list.append(df)
    print('Length after filtering {}'.format(len(df.index)))

veiculos = pd.concat(df_list).rename(index=str, columns=columns_rename).astype(columns_types)


veiculos.line_id = veiculos.line_id.apply(str)
veiculos.line_id = veiculos.line_id.astype('category')
veiculos['date_time'] = veiculos['date_time'].apply(parse_brazilian_datetime)
print(veiculos.dtypes)
veiculos.head()

In [ ]:
#print('After line filtering')
veiculos['coordinates'] = veiculos.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
veiculos = gpd.GeoDataFrame(veiculos, geometry='coordinates', crs=gps_datum)
veiculos = veiculos.to_crs(sirgas_utm_22s_datum)
veiculos['date_time_shifted'] = veiculos.groupby(by=['line_id','vehicle_id'])['date_time'].shift(1)
veiculos['coordinates_shifted'] = veiculos.groupby(by=['line_id','vehicle_id'])['coordinates'].shift(1)
veiculos.dropna(axis=0, how='any', inplace=True)
veiculos['line_string'] = veiculos.apply(lambda row: points_to_linestring([row['coordinates'],row['coordinates_shifted']]), axis=1)
veiculos = veiculos.set_geometry('line_string')
veiculos['line_string_length'] = veiculos['line_string'].length
veiculos['time_dif'] = veiculos['date_time_shifted'] - veiculos['date_time']
veiculos['km_per_hour'] = (veiculos['line_string_length']/1000)/ (veiculos['time_dif'].dt.total_seconds()/3600)
print(veiculos.dtypes)
print('km_per_hour analysis:\n {}\n'.format(veiculos.km_per_hour.describe()))
print('distance analysis:\n {}\n'.format(veiculos.line_string_length.describe()))
veiculos = veiculos[['date','line_id','vehicle_id','date_time','line_string','line_string_length','time_dif','km_per_hour']]
veiculos.head()

In [ ]:
for selected_bus_line in chosen_bus_lines:
    print('The vehicles operating in the line {} are {}.'
          .format(
              selected_bus_line,
              veiculos.vehicle_id[veiculos.line_id == selected_bus_line].unique()
          )
         )

In [ ]:
fig = veiculos['line_string'].plot(edgecolor='red', color='red')
print('The Map')
mplleaflet.show(fig=fig.figure, crs=sirgas_utm_22s_datum, tiles='cartodb_positron', path='hello.html')

In [ ]:
###### LINHAS ######
columns_rename = {
    'CATEGORIA_SERVICO':'category',
    'COD': 'line_id',
    'NOME': 'line_name',
    'NOME_COR': 'color',
    'SOMENTE_CARTAO': 'only_card'
}

columns_types = {
    'category': 'category',
    'line_id': 'category',
    'line_name': 'category',
    'color': 'category',
    'only_card': 'category'
}

filtered_files = files_filtered_by_date[files_filtered_by_date['file_type'] == 'linhas']

df_list = []
print("Using the following files:")
for index, row in filtered_files.iterrows():
    file = row['file_path']
    print(file)
    date = row['date']
    df = pd.read_json(orient='records',path_or_buf=file)
    df['date'] = date
    df['COD'] = df['COD'].apply(str)
    df_list.append(df)
    
linhas = pd.concat(df_list).rename(index=str, columns=columns_rename).astype(columns_types)

# linhas.reset_index(level=-1, inplace=True)
'''
linhas.set_index(
    keys=['date','line_id'],
    drop=False,
    append=False,
    inplace=True,
    verify_integrity=True
)
'''

linhas = linhas[linhas['line_id'].isin(chosen_bus_lines)]
linhas

In [ ]:
##### PONTOS LINHA #####

columns_rename = {
    'COD':'line_id',
    'GRUPO': 'group_id',
    'ITINERARY_ID': 'itinerary_id',
    'LAT': 'lat',
    'LON': 'lon',
    'NOME': 'stop_name',
    'NUM': 'stop_id',
    'SENTIDO': 'way',
    'SEQ': 'stop_sequence',
    'TIPO': 'building'
}

columns_types = {
    'line_id': 'category',
    'group_id': 'category',
    'itinerary_id': 'category',
    'lat': 'float64',
    'lon': 'float64',
    'stop_name': 'category',
    'stop_id': 'object',
    'way': 'category',
    'stop_sequence': 'uint8',
    'building': 'category'
}

filtered_files = files_filtered_by_date[files_filtered_by_date['file_type'] == 'pontosLinha']

df_list = []
print("Using the following files:")
for index, row in filtered_files.iterrows():
    file = row['file_path']
    print(file)
    date = row['date']
    df = pd.read_json(orient='records',path_or_buf=file)
    df['date'] = date
    df['COD'] = df['COD'].apply(str)
    df_list.append(df)
    
pontosLinha = pd.concat(df_list).rename(index=str, columns=columns_rename)
pontosLinha = pontosLinha.astype(columns_types)
pontosLinha['stop_id'] = pontosLinha['stop_id'].apply(str)
pontosLinha['stop_id'] = pontosLinha['stop_id'].astype('category')

'''
pontosLinha.set_index(
    keys=['date','line_id','itinerary_id','stop_sequence','way'],
    drop=False,
    append=False,
    inplace=True,
    verify_integrity=True
)
'''

pontosLinha = pontosLinha[pontosLinha['line_id'].isin(chosen_bus_lines)]

pontosLinha.sort_values(by=['date','line_id','itinerary_id','way','stop_sequence',],inplace=True)
pontosLinha['stop_coord'] = pontosLinha.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
pontosLinha = gpd.GeoDataFrame(pontosLinha, geometry='stop_coord', crs=gps_datum)
pontosLinha = pontosLinha.to_crs(sirgas_utm_22s_datum)
pontosLinha = pontosLinha.reindex(columns=['date','line_id','itinerary_id','way','stop_sequence','stop_id','stop_coord','stop_name','building','group_id'])
pontosLinha['stop_buffer'] = pontosLinha['stop_coord'].buffer(buffer_radius)
uniqueItineraries = pontosLinha['itinerary_id'].unique()
print(uniqueItineraries)
#print('Counting frequency of stop_ids:')
#print(pontosLinha['stop_id'].value_counts())
bus_stops = pontosLinha.drop_duplicates(subset=['date','line_id','stop_id'], keep='first', inplace=False)
bus_stops = bus_stops[bus_stops['line_id'].isin(chosen_bus_lines)]
#bus_stops = bus_stops[['date','line_id','stop_id','stop_coord','stop_name','building','group_id','stop_buffer']]
print('Number of repeated rows by {}\n************************'.format(bus_stops.groupby(['line_id','itinerary_id','stop_sequence','stop_id'],as_index=False).size()))
bus_stops.drop_duplicates(subset=['line_id','itinerary_id','stop_sequence','stop_id'],keep='first',inplace=True)
bus_stops.drop(labels=['date','line_id','itinerary_id','way','group_id'],axis=1,inplace=True)
bus_stops

In [ ]:
bus_stops = gpd.GeoDataFrame(bus_stops, geometry='stop_buffer', crs=sirgas_utm_22s_datum)
fig = bus_stops['stop_buffer'].plot(ax=fig, edgecolor='green', color='green')
# Displaying Map
print('The Map')
mplleaflet.show(fig=fig.figure, crs=sirgas_utm_22s_datum, tiles='cartodb_positron',path='stops-path.html')

In [ ]:
vehicle_over_stop = gpd.sjoin(veiculos, bus_stops, how='inner', op='intersects', lsuffix='vehi', rsuffix='stop')
vehicle_over_stop.head()

# Fazer loop sobre vehicles separando por date e vehicle_id
# 

In [ ]:
vehicle_over_stop = gpd.GeoDataFrame(vehicle_over_stop, geometry='line_string', crs=sirgas_utm_22s_datum)
vehicle_over_stop['distance_from_stop'] = gpd.GeoSeries(vehicle_over_stop['line_string'], crs=sirgas_utm_22s_datum).distance(gpd.GeoSeries(vehicle_over_stop['stop_coord'], crs=sirgas_utm_22s_datum))
vehicle_over_stop.head()

In [ ]:
vehicle_over_stop['date_time_group'] = np.uint16(0)
date_time_column_id = vehicle_over_stop.columns.get_loc('date_time')
date_time_group_column_id = vehicle_over_stop.columns.get_loc('date_time_group')
group_id = np.uint16(0)
for i in range(0, len(vehicle_over_stop)):
    try:
        # Pega o tempo anterior
        # Compara com o atual
        # Se a diferença dos dois for 
        # menor que certa threshold, seta o grupo group_id nela
        # maior que certa threshold, atualiza group_id + 1 e seta o group_id nela
        previous_date_time = vehicle_over_stop.iat[i-1, date_time_column_id]
        current_date_time = vehicle_over_stop.iat[i, date_time_column_id]
        absolute_time_dif_seconds = np.absolute(current_date_time - previous_date_time)/np.timedelta64(1, 's')
        # print('Absolute time dif {}'.format(absolute_time_dif_seconds))
        if absolute_time_dif_seconds > time_dif_seconds_threshold:
            group_id = group_id + 1
        vehicle_over_stop.iat[i, date_time_group_column_id] = group_id
        # print('Group id {}'.format(group_id))
    except Exception as e:
        print(e)
vehicle_over_stop
vehicle_over_stop

In [ ]:
vehicle_over_stop_grouped = vehicle_over_stop.loc[vehicle_over_stop.groupby('date_time_group')['distance_from_stop'].idxmin()]
vehicle_over_stop_grouped.sort_values(by=['date_time'],inplace=True)
vehicle_over_stop_grouped

In [ ]:
# Importing and cleaning table shapeLinha

columns_rename = {
    'COD': 'line_id',
    'LAT': 'lat',
    'LON': 'lon',
    'SHP': 'shape_id'
}

columns_types = {
    'line_id': 'category',
    'lat': 'float64',
    'lon': 'float64',
    'shape_id': 'uint32'
}

filtered_files = files_filtered_by_date[files_filtered_by_date['file_type'] == 'shapeLinha']

df_list = []
print("Using the following files:")
for index, row in filtered_files.iterrows():
    file = row['file_path']
    print(file)
    date = row['date']
    df = pd.read_json(orient='records',path_or_buf=file)
    df['date'] = date
    df['COD'] = df['COD'].apply(str)
    # Changes the index name to point_sequence, because the order the points are presented in the archive matters
    df.index.name = 'point_sequence'
    df.reset_index(level=0, inplace=True)
    df_list.append(df)
    
shapeLinha = pd.concat(df_list).rename(index=str, columns=columns_rename).astype(columns_types)



shapeLinha = shapeLinha[shapeLinha['line_id'].isin(chosen_bus_lines)]

# Creating the georefferenced shape
shapeLinha['coordinates'] = shapeLinha.apply(lambda row: Point(row['lon'], row['lat']), axis=1)

# Gets a list of unique shape_ids
unique_shapes = shapeLinha['shape_id'].unique()
# Gets a list of unique line_ids
unique_lines = shapeLinha['line_id'].unique()
# Gets a list of unique dates
unique_dates = shapeLinha['date'].unique()

shapeLinha['coordinates_shifted'] = shapeLinha.groupby(by=['date','line_id','shape_id'])['coordinates'].shift(1)

# Deleting rows with NaN 
shapeLinha.dropna(axis=0, how='any', inplace=True)

# Creating line_string column
shapeLinha['line_string'] = shapeLinha.apply(lambda row: points_to_linestring([row['coordinates'],row['coordinates_shifted']]), axis=1)
shapeLinha = gpd.GeoDataFrame(shapeLinha, geometry='line_string', crs=gps_datum)
shapeLinha = shapeLinha.to_crs(sirgas_utm_22s_datum)
shapeLinha['distance'] = shapeLinha.geometry.length
print('Number of unique shapes {}. The shape ids are: {}.'.format(len(unique_shapes), unique_shapes))
print('Number of unique dates {}. The dates are {}'.format(len(unique_dates),unique_dates))

In [ ]:
# Defining Plot
plotColors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']

# Creating the index
'''
shapeLinha.set_index(
    keys=['date','line_id','shape_id','point_sequence'],
    drop=False,
    append=False,
    inplace=True,
    verify_integrity=True
)
'''

# Filtering using indexes
# fig = shapeLinha.loc['2018-10-01',chosen_bus_line,2778,:].plot(edgecolor='r', color='r')
# fig = shapeLinha.loc['2018-10-01',chosen_bus_line,1785,:].plot(ax=fig, edgecolor='b', color='b')
# fig = shapeLinha.loc['2018-10-02',chosen_bus_line,2778,:].plot(ax=fig, edgecolor='g', color='g')
# Filtering using regular columns
count = 0
for date in unique_dates:
    for line in unique_lines:
        for shape in unique_shapes:
            current_color = 'C'+str(count)
            if fig is None:
                fig = shapeLinha[
                    (shapeLinha['date'] == date) & 
                    (shapeLinha['line_id'] == line) & 
                    (shapeLinha['shape_id'] == shape)
                ].plot(edgecolor=current_color, color=current_color)
            else:
                fig = shapeLinha[
                    (shapeLinha['date'] == date) & 
                    (shapeLinha['line_id'] == line) & 
                    (shapeLinha['shape_id'] == shape)
                ].plot(ax=fig,edgecolor=current_color, color=current_color)
            count = count + 1
            print('Date {}, Line {}, Shape {} is on the map with the color {}'.format(date, line, shape, current_color))

print('{} shapes were plotted on the map.'.format(count))

In [ ]:
# Displaying Map
#print('The Map')
#mplleaflet.display(fig=fig.figure, crs=shapeLinha.crs, tiles='cartodb_positron')

In [ ]:
# Tabela Veiculo

''' NOT USED! DID NOT WORK USING APPLY IN THE DATAFRAME
def fix_dates(str_date, str_time, time_threshold):
    """
    Returns a datetime64 datetime with the concatenation of str_date and str_time
    str_date is the date as a string
    str_time is a time as a string
    time_threshold is a time limit. str_time is lower than the threshold, adds 1 day to the str_date
    """
    pdate = datetime.strptime(str_date, '%Y-%m-%d').date() 
    ptime = datetime.strptime(str_time, '%H:%M').time()
    print('pdate {} ptime {} time threshold {}'.format(pdate, ptime, time_threshold))
    if ptime < time_threshold:
        pdate = pdate + timedelta(days=1)
    return np.datetime64(datetime.combine(pdate, ptime))
'''

columns_rename = {
    'COD_LINHA': 'line_id',
    'COD_PONTO': 'stop_id',
    'HORARIO': 'scheduled_time',
    'NOME_LINHA': 'line_name',
    'TABELA': 'route_table',
    'VEICULO': 'vehicle_id'
}

columns_types = {
    'line_id': 'category',
    'stop_id': 'category',
    'line_name': 'category',
    'route_table': 'category',
    'vehicle_id': 'category'
}

filtered_files = files_filtered_by_date[files_filtered_by_date['file_type'] == 'tabelaVeiculo']

df_list = []

print("Using the following files:")
for index, row in filtered_files.iterrows():
    file = row['file_path']
    print(file)
    date = row['date']
    df = pd.read_json(orient='records',path_or_buf=file)
    df['date'] = date
    df['COD_LINHA'] = df['COD_LINHA'].apply(str)
    df_list.append(df)
    
tabelaVeiculo = pd.concat(df_list).rename(index=str, columns=columns_rename).astype(columns_types)

unique_stop_ids = tabelaVeiculo['stop_id'].unique()

tabelaVeiculo = tabelaVeiculo[tabelaVeiculo['line_id'].isin(chosen_bus_lines)]
tabelaVeiculo = tabelaVeiculo[tabelaVeiculo['vehicle_id'].isin(chosen_vehicles)]

first_day = np.datetime64('1900-01-01')
tabelaVeiculo['fixed_scheduled_time'] = pd.to_datetime(
    tabelaVeiculo['scheduled_time'], 
    format='%H:%M'
).apply(
    lambda x: x - first_day
)
tabelaVeiculo['fixed_scheduled_time'] = tabelaVeiculo['fixed_scheduled_time'] + tabelaVeiculo['date']

print(tabelaVeiculo.dtypes)

tabelaVeiculo = tabelaVeiculo[['date','line_id','vehicle_id','fixed_scheduled_time','scheduled_time','stop_id','route_table']]
print('Number of repeated rows by {}\n************************'.format(tabelaVeiculo.groupby(['line_id','vehicle_id','stop_id','scheduled_time','route_table'],as_index=False).size()))
tabelaVeiculo

In [ ]:
vehicle_over_stop_grouped.index.names = ['vos_index']
vehicle_over_stop_grouped.reset_index(inplace=True)
vehicle_over_stop_grouped.head()

In [ ]:
tv_vos = pd.merge(
    vehicle_over_stop_grouped,tabelaVeiculo,
    how='inner',
    left_on=['stop_id'],
    right_on=['stop_id'],
    copy=False
)

tv_no_schedule = vehicle_over_stop_grouped[vehicle_over_stop_grouped.stop_id.isin(tabelaVeiculo.stop_id)]

print(tv_vos.dtypes)
print('--')
print(tv_no_schedule.dtypes)

In [ ]:
tv_vos['t_scheduled'] = tv_vos['fixed_scheduled_time'].dt.time.apply(lambda x: pd.Timedelta(str(x)))
tv_vos['t_real'] = tv_vos['date_time'].dt.time.apply(lambda x: pd.Timedelta(str(x)))
tv_vos['delay_amount'] =  tv_vos.t_real - tv_vos.t_scheduled
tv_vos['abs_delay_amount'] =  np.abs(tv_vos['delay_amount'])
tv_vos.sort_values(by=['vos_index','abs_delay_amount'],ascending=True,inplace=True,na_position='last')
tv_vos

In [ ]:
clean_tv_vos = tv_vos.loc[tv_vos.groupby(['vos_index'])['abs_delay_amount'].idxmin()]
clean_tv_vos[['vos_index','date_x','date_time','t_real','t_scheduled','delay_amount','abs_delay_amount','distance_from_stop']]

In [ ]:
clean_tv_vos[['vos_index','date_x','date_time','t_real','t_scheduled','delay_amount','abs_delay_amount','distance_from_stop']].describe()

In [ ]:
tt = pd.DataFrame({
    'dt1':['2018-01-01 12:22','2018-01-01 23:22','2018-01-01 12:00','2018-01-01 11:22'],
    'dt2':['2018-01-01 12:26','2018-01-01 23:42','2018-01-01 12:20','2018-01-01 11:52']
})
tt['dt1'] = pd.to_datetime(tt['dt1'])
tt['dt2'] = pd.to_datetime(tt['dt2'])
tt['dt'] = tt['dt1'].dt.time
tt['x'] = tt['dt1'].dt.time.apply(lambda x: pd.Timedelta(str(x)))
print(tt.dtypes)
tt